<a target="_blank" href="https://colab.research.google.com/github/gox6/colab-demos/blob/main/wip/polars-a-modern-dataframe-library.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

<html>
<body>
    <div style="display: flex;">
        <a href="https://towardsdatascience.com/evaluate-rags-rigorously-or-perish-54f790557357" style="width: 45em; height: 15em; background-color: white; padding-left: 1em; display: flex; flex-direction: column; justify-content: center; color: inherit; text-decoration: none;">
            <p style="margin: 0; font-size: 1.5em; color: darkblue; font-weight: bold;">The notebook contains the code for Medium article</p>
            <p style="margin: 0; font-weight: bold; font-size: 1.5em; color: #3D4849;">Invent the title</p>
        </a>
        <a href="https://towardsdatascience.com/evaluate-rags-rigorously-or-perish-54f790557357" style="width: 30em; height: 15em;">
            <img src="https://images.unsplash.com/photo-1522991150037-09a660b94ba4?q=80&w=1470&auto=format&fit=crop&ixlib=rb-4.0.3&ixid=M3wxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8fA%3D%3D" style="width: 100%; height: 100%; object-fit: cover;">
        </a>
    </div>
</body>
</html>

# Project Setup

In [11]:
# Installing Python packages quietly

!pip install --quiet \
  polars \
  1> /dev/null

In [12]:
# Importing the packages
from getpass import getpass
import os

import polars as pl

In [13]:
# Managing secrets
# - If using Colab please use Colab Secrets
# - If running outside Colab please provide secrets as environmental variables
COLAB = os.getenv("COLAB_RELEASE_TAG") is not None

if COLAB:
    from google.colab import userdata, data_table

    # Secrets
    OPENAI_API_KEY = userdata.get("OPENAI_API_KEY")
    os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
    runtime_info = "Colab runtime"

    # Enabling Colab's data formatter for pandas
    data_table.enable_dataframe_formatter()
elif OPENAI_API_KEY := os.environ.get("OPENAI_API_KEY"):
    # Secrets
    runtime_info = "Non Colab runtime"
else:
    OPENAI_API_KEY = getpass("OPENAI_API_KEY")
    os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
    runtime_info = "Non Colab runtime"

print(runtime_info)

Colab runtime


In [23]:
# You need free tier GCP account
from google.colab import auth
auth.authenticate_user()

In [42]:

from google.cloud import bigquery
client = bigquery.Client(project='handy-buttress-422914-t8')

In [55]:

datasets = list(client.list_datasets())  # Make an API request.
project = client.project

if datasets:
    print("Datasets in project {}:".format(project))
    for dataset in datasets:
        print("\t{}".format(dataset.dataset_id))
else:
    print("{} project does not contain any datasets.".format(project))

Datasets in project bigquery-public-data:
	america_health_rankings
	austin_311
	austin_bikeshare
	austin_crime
	austin_incidents
	austin_waste
	baseball
	bbc_news
	bigqueryml_ncaa
	bitcoin_blockchain
	blackhole_database
	blockchain_analytics_ethereum_mainnet_us
	bls
	bls_qcew
	breathe
	broadstreet_adi
	catalonian_mobile_coverage
	catalonian_mobile_coverage_eu
	census_bureau_acs
	census_bureau_construction
	census_bureau_international
	census_bureau_usa
	census_opportunity_atlas
	census_utility
	cfpb_complaints
	chicago_crime
	chicago_taxi_trips
	clemson_dice
	cloud_storage_geo_index
	cms_codes
	cms_medicare
	cms_synthetic_patient_data_omop
	country_codes
	covid19_aha
	covid19_covidtracking
	covid19_ecdc
	covid19_ecdc_eu
	covid19_genome_sequence
	covid19_geotab_mobility_impact
	covid19_geotab_mobility_impact_eu
	covid19_google_mobility
	covid19_google_mobility_eu
	covid19_govt_response
	covid19_italy
	covid19_italy_eu
	covid19_jhu_csse
	covid19_jhu_csse_eu
	covid19_nyt
	covid19_open_dat

In [68]:
# Perform a query.
%%time
client = bigquery.Client(project='handy-buttress-422914-t8')
QUERY = (
    """SELECT * FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
    LIMIT 10000000"""
    )
query_job = client.query(QUERY)  # API request
rows = query_job.result()  # Waits for query to finish

df = pl.from_arrow(rows.to_arrow())

CPU times: user 20.4 s, sys: 18.5 s, total: 38.8 s
Wall time: 32.1 s


In [67]:
df.describe()

describe,unique_key,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,fare,tips,tolls,extras,trip_total,payment_type,company,pickup_latitude,pickup_longitude,pickup_location,dropoff_latitude,dropoff_longitude,dropoff_location
str,str,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,str,f64,f64,str,f64,f64,str
"""count""","""1000000""","""1000000""","""1000000""","""999998""",999853.0,999963.0,328025.0,347805.0,422165.0,482220.0,999943.0,999943.0,764017.0,999943.0,999943.0,"""1000000""","""474449""",422182.0,422182.0,"""422182""",483949.0,483949.0,"""483949"""
"""null_count""","""0""","""0""","""0""","""2""",147.0,37.0,671975.0,652195.0,577835.0,517780.0,57.0,57.0,235983.0,57.0,57.0,"""0""","""525551""",577818.0,577818.0,"""577818""",516051.0,516051.0,"""516051"""
"""mean""",null,null,null,null,829.627538,4.308541,1.7031e10,1.7031e10,21.926488,23.550624,12.678485,1.154758,0.011009,0.937429,14.826923,null,null,41.904451,-87.658214,null,41.907358,-87.667955,null
"""std""",null,null,null,null,1600.702144,11.452632,340391.465488,343293.185461,19.592892,21.028473,57.1031,2.730116,1.704209,6.726246,58.374663,null,null,0.036569,0.069566,null,0.045144,0.077432,null
"""min""","""000004c5d7cfd4…","""0005e9157e3cfe…","""2013-04-11 16:…","""2013-04-11 16:…",0.0,0.0,1.7031e10,1.7031e10,1.0,1.0,0.0,0.0,0.0,0.0,0.0,"""Cash""","""1085 - 72312 N…",41.67382,-87.913625,"""POINT (-87.534…",41.660136,-87.913625,"""POINT (-87.534…"
"""25%""",null,null,null,null,300.0,0.8,1.7031e10,1.7031e10,8.0,8.0,5.25,0.0,0.0,0.0,6.05,null,null,41.884987,-87.655998,null,41.880994,-87.664078,null
"""50%""",null,null,null,null,540.0,1.7,1.7031e10,1.7031e10,8.0,21.0,7.65,0.0,0.0,0.0,9.0,null,null,41.892658,-87.632746,null,41.895033,-87.637844,null
"""75%""",null,null,null,null,983.0,4.5,1.7032e10,1.7032e10,32.0,32.0,13.0,2.0,0.0,1.0,14.65,null,null,41.922083,-87.626211,null,41.936237,-87.626215,null
"""max""","""ffffb296eb928b…","""fff84aa08ac788…","""2022-10-30 12:…","""2022-10-30 12:…",86388.0,2151.86,1.7032e10,1.7032e10,77.0,77.0,9140.71,800.0,999.99,5560.55,9190.71,"""Unknown""","""Yellow Cab""",42.021224,-87.534903,"""POINT (-87.913…",42.021224,-87.534903,"""POINT (-87.913…"


In [60]:
import polars as pl
from google.cloud import bigquery

client = bigquery.Client()

# Perform a query.
QUERY = (
    'SELECT * FROM `bigquery-public-data.usa_names.usa_1910_2013` LIMIT 100')
query_job = client.query(QUERY)  # API request
rows = query_job.result()  # Waits for query to finish

df = pl.from_arrow(rows.to_arrow())

NotFound: 404 POST https://bigquery.googleapis.com/bigquery/v2/projects//jobs?prettyPrint=false: Request couldn't be served.

Location: None
Job ID: 45407d9a-8fce-48a3-8925-b764285b9980


In [58]:
import pandas as pd

# https://cloud.google.com/resource-manager/docs/creating-managing-projects
project_id = 'handy-buttress-422914-t8'
sample_count = 2000

row_count = pd.io.gbq.read_gbq('''
  SELECT
    COUNT(*) as total
  FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
''', project_id=project_id).total[0]

df = pd.io.gbq.read_gbq(f'''
  SELECT
    *
  FROM
    `bigquery-public-data.samples.gsod`
  WHERE RAND() < {sample_count}/{row_count}
''', project_id=project_id)

print(f'Full dataset has {row_count} rows')

Full dataset has 102589284 rows


<html>
<body>
    <div style="display: flex;">
            <a href="https://medium.com/@jgrygolec" style="width: 45em; height: 15em; background-color: white; padding-left: 1em; display: flex; flex-direction: column; justify-content: center; color: inherit; text-decoration: none;">
            <p style="margin: 0; font-size: 1.5em; color: #3D4849;">Thank you for attention!</p>
            <p style="margin: 0; font-size: 1.5em; color: #3D4849; font-weight: bold;">Click to see more of my articles on Medium.</p>
            <p style="margin: 0; font-size: 1.5em; color: #3D4849;">BR,</p>
            <p style="margin: 0; font-size: 1.5em; color: #3D4849;">Jarek Grygolec</p>
        </a>
        <a href="https://medium.com/@jgrygolec" style="width: 30em; height: 15em; background-repeat: no-repeat; background-size: cover; background-position: center;">
                    <img src="https://miro.medium.com/v2/resize:fit:3992/0*WCQwsoZC0FA2-haq" style="width: 100%; height: 100%; object-fit: cover;"></a>
    </div>
</body>
</html>